### Projeto de Predição da Evolução do Índice IFIX

O Índice IFIX é o resultado de uma carteira teórica de ativos de fundos imobiliários. Seu objetivo é ser o indicador do desempenho médio das cotações dos fundos imobiliários negociados nos mercados de bolsa e de balcão organizado da B3.

> Os dados do índice foram coletados do site da B3 e podem ser acessados [neste link](https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-de-segmentos-e-setoriais/indice-fundos-de-investimentos-imobiliarios-ifix-estatisticas-historicas.htm).

Para as variáveis preditoras, foram usados os dados históricos das taxas CDI e SELIC. 

* Taxa SELIC: é gerada através da média ponderada das operações diárias titulos públicos, sendo as taxas praticadas balizadas pela SELIC meta.

* CDI: de forma semelhante, é a estabelecido pela média das taxas praticadas entre dos empréstimos diários interbancários. É também balizada pela SELIC meta.

> Dados da SELIC e CDI podem ser obitidos também no site da B3 [neste link](http://estatisticas.cetip.com.br/astec/series_v05/paginas/lum_web_v05_template_informacoes_di.asp?str_Modulo=completo&int_Idioma=1&int_Titulo=6&int_NivelBD=2).




In [0]:
from pyspark.sql import SparkSession #importa a biblioteca que cria a seção do spark

In [0]:
#inicia a seção para a utilização do spark
spark = SparkSession.builder.appName("Regressao IFIX").getOrCreate() #cria a seção caso não exista ou obtém a já criada

In [0]:
%fs ls /FileStore/tables

path,name,size,modificationTime
dbfs:/FileStore/tables/HisoricoCDI.csv,HisoricoCDI.csv,115462,1710016875000
dbfs:/FileStore/tables/HistoricoCDI.csv,HistoricoCDI.csv,115462,1710017252000
dbfs:/FileStore/tables/HistoricoSELIC.csv,HistoricoSELIC.csv,43851,1710016875000
dbfs:/FileStore/tables/IfixEvolucaoMensal.csv,IfixEvolucaoMensal.csv,2347,1710016875000
dbfs:/FileStore/tables/adult_data.csv,adult_data.csv,5608318,1709852380000
dbfs:/FileStore/tables/iris_bezdekIris.csv,iris_bezdekIris.csv,4551,1709994811000
dbfs:/FileStore/tables/regressaoLinear.csv,regressaoLinear.csv,564,1709993226000
dbfs:/FileStore/tables/temperature.csv,temperature.csv,4075992,1709677144000
dbfs:/FileStore/tables/temperatureAmericas.csv,temperatureAmericas.csv,14644660,1709685210000
dbfs:/FileStore/tables/temperatureMajorCity.csv,temperatureMajorCity.csv,14138385,1709678535000


In [0]:
 # diretórios dos arquivos a serem utilizados
historicoCDI="/FileStore/tables/HistoricoCDI.csv"
historicoSELIC="/FileStore/tables/HistoricoSELIC.csv"
historicoIFIX="/FileStore/tables/IfixEvolucaoMensal.csv"

#### Dataset CDI

In [0]:
#lendo arquivos armazenados CSV com o esquema definido
cdi = spark.read.format('csv').options(inferSchema=True, header='true',delimiter=';').load(historicoCDI)

In [0]:
cdi.show(5)

+----------+-------------+-------+-----+------------+
|      Data|Nr. Operações| Volume|Média|Fator Diário|
+----------+-------------+-------+-----+------------+
|2013-01-02|           15|1892473| 6,92|  1,00026555|
|2013-01-03|           16|2007587| 6,92|  1,00026555|
|2013-01-04|           17|2138985| 6,93|  1,00026593|
|2013-01-07|           18|2208986| 6,92|  1,00026555|
|2013-01-08|           18|2246656| 6,91|  1,00026518|
+----------+-------------+-------+-----+------------+
only showing top 5 rows



In [0]:
cdi.printSchema()

root
 |-- Data: date (nullable = true)
 |-- Nr. Operações: integer (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Média: string (nullable = true)
 |-- Fator Diário: string (nullable = true)



In [0]:
import pyspark.sql.functions as F
cdi = cdi.select(F.col('Média').alias("CDI"), F.col('Data').alias("Data"))  # trocando nomes do cabaçalho e selecionando as colunas que preciso

In [0]:
from pyspark.sql.functions import dayofmonth, month, year, last_day
# Adicione uma nova coluna contendo o último dia de cada mês
cdiData = cdi.withColumn("ultimo_dia", last_day("Data"))
# Filtrando pelo último dia de cada mês
cdiData = cdiData.filter(dayofmonth("Data") == dayofmonth("ultimo_dia"))

cdiData.show(5)

+----+----------+----------+
| CDI|      Data|ultimo_dia|
+----+----------+----------+
|6,95|2013-01-31|2013-01-31|
|6,98|2013-02-28|2013-02-28|
|7,23|2013-04-30|2013-04-30|
|7,72|2013-05-31|2013-05-31|
|8,22|2013-07-31|2013-07-31|
+----+----------+----------+
only showing top 5 rows



In [0]:
cdiData = cdiData.select('Data', "CDI")
cdiData.show(5)

+----------+----+
|      Data| CDI|
+----------+----+
|2013-01-31|6,95|
|2013-02-28|6,98|
|2013-04-30|7,23|
|2013-05-31|7,72|
|2013-07-31|8,22|
+----------+----+
only showing top 5 rows



In [0]:
cdiFinal = cdiData.withColumn("CDI", F.regexp_replace(F.col("CDI"), "[,]", "."))  #troca o valor de "," para "." 

In [0]:
cdiFinal.show(5)

+----------+----+
|      Data| CDI|
+----------+----+
|2013-01-31|6.95|
|2013-02-28|6.98|
|2013-04-30|7.23|
|2013-05-31|7.72|
|2013-07-31|8.22|
+----------+----+
only showing top 5 rows



In [0]:
cdiFinal.printSchema()

root
 |-- Data: date (nullable = true)
 |-- CDI: string (nullable = true)



In [0]:
# Trocando CDI para tipo double
from pyspark.sql.functions import col
cdiConcat = cdiFinal.withColumn("CDI", col("CDI").cast("double"))

In [0]:
cdiConcat.printSchema()

root
 |-- Data: date (nullable = true)
 |-- CDI: double (nullable = true)



In [0]:
display(cdiConcat)

Data,CDI
2013-01-31,6.95
2013-02-28,6.98
2013-04-30,7.23
2013-05-31,7.72
2013-07-31,8.22
2013-09-30,8.71
2013-10-31,9.32
2013-12-31,9.77
2014-01-31,10.27
2014-02-28,10.59


Databricks visualization. Run in Databricks to view.

#### Dataset IFIX

In [0]:
# Importanto dataset do IFIX
ifix = spark.read.format('csv').options(inferSchema=True, header='true',delimiter=';').load(historicoIFIX)

In [0]:
ifix.show(5)

+---+----+--------+
|Mês| Ano|   Valor|
+---+----+--------+
|  1|2013|1.613,76|
|  2|2013|1.597,88|
|  3|2013|1.579,32|
|  4|2013|1.549,00|
|  5|2013|1.549,36|
+---+----+--------+
only showing top 5 rows



In [0]:
# Concatenando das colunas Mês e Ano e ajustando ao modelo yyyy-MM-dd
from pyspark.sql.functions import concat, lit, to_date
ifixData = ifix.withColumn("Data", to_date(concat(col("Ano"), lit("-"), col("Mês"), lit("-01"))))


In [0]:
ifixData.show(5)

+---+----+--------+----------+
|Mês| Ano|   Valor|      Data|
+---+----+--------+----------+
|  1|2013|1.613,76|2013-01-01|
|  2|2013|1.597,88|2013-02-01|
|  3|2013|1.579,32|2013-03-01|
|  4|2013|1.549,00|2013-04-01|
|  5|2013|1.549,36|2013-05-01|
+---+----+--------+----------+
only showing top 5 rows



In [0]:
# Alterando a coluna Valor para nome IFIX
ifixData = ifixData.select(F.col('Valor').alias("IFIX"), F.col('Data').alias("Data"))
ifixData.show(5)

+--------+----------+
|    IFIX|      Data|
+--------+----------+
|1.613,76|2013-01-01|
|1.597,88|2013-02-01|
|1.579,32|2013-03-01|
|1.549,00|2013-04-01|
|1.549,36|2013-05-01|
+--------+----------+
only showing top 5 rows



In [0]:
# Retirando o ponto da casa de milhares
from pyspark.sql.functions import regexp_replace
ifixFinal = ifixData.withColumn("IFIX", regexp_replace("IFIX", "\.", ""))
ifixFinal.show(5)

+-------+----------+
|   IFIX|      Data|
+-------+----------+
|1613,76|2013-01-01|
|1597,88|2013-02-01|
|1579,32|2013-03-01|
|1549,00|2013-04-01|
|1549,36|2013-05-01|
+-------+----------+
only showing top 5 rows



In [0]:
# Trocando virgula por ponto
ifixFinal = ifixFinal.withColumn("IFIX", F.regexp_replace(F.col("IFIX"), "[,]", "."))
ifixFinal.show(5)

+-------+----------+
|   IFIX|      Data|
+-------+----------+
|1613.76|2013-01-01|
|1597.88|2013-02-01|
|1579.32|2013-03-01|
|1549.00|2013-04-01|
|1549.36|2013-05-01|
+-------+----------+
only showing top 5 rows



In [0]:
ifixFinal.printSchema()

root
 |-- Mês: integer (nullable = true)
 |-- Ano: integer (nullable = true)
 |-- Valor: string (nullable = true)
 |-- Data: date (nullable = true)



In [0]:
ifixConcat = ifixFinal.withColumn("IFIX", col("IFIX").cast("Double")) # trocando valor string para float

In [0]:
ifixConcat.printSchema()

root
 |-- IFIX: double (nullable = true)
 |-- Data: date (nullable = true)



In [0]:
ifixConcat.show(5)

+-------+----------+
|   IFIX|      Data|
+-------+----------+
|1613.76|2013-01-01|
|1597.88|2013-02-01|
|1579.32|2013-03-01|
| 1549.0|2013-04-01|
|1549.36|2013-05-01|
+-------+----------+
only showing top 5 rows



In [0]:
display(ifixConcat)

IFIX,Data
1613.76,2013-01-01
1597.88,2013-02-01
1579.32,2013-03-01
1549.0,2013-04-01
1549.36,2013-05-01
1437.76,2013-06-01
1427.11,2013-07-01
1375.07,2013-08-01
1400.93,2013-09-01
1423.1,2013-10-01


Databricks visualization. Run in Databricks to view.

In [0]:
ifixConcat.describe().show()

+-------+-----------------+
|summary|             IFIX|
+-------+-----------------+
|  count|              135|
|   mean|2243.024074074075|
| stddev|647.2107060950273|
|    min|          1282.04|
|    max|          3382.28|
+-------+-----------------+



#### Dataset SELIC  

In [0]:
selic = spark.read.format('csv').options(inferSchema=True, header='true',delimiter=';').load(historicoSELIC)

In [0]:
selic.show(5)

+----------+----------+
|      Data|Taxa SELIC|
+----------+----------+
|2014-01-02|       9,9|
|2014-01-03|       9,9|
|2014-01-06|       9,9|
|2014-01-07|       9,9|
|2014-01-08|       9,9|
+----------+----------+
only showing top 5 rows



In [0]:
selic.printSchema()

root
 |-- Data: date (nullable = true)
 |-- Taxa SELIC: string (nullable = true)



In [0]:
# Renomeando a coluna Taxa SELIC para apenas SELIC
selic = selic.select(F.col('Taxa SELIC').alias("SELIC"), F.col('Data').alias("Data"))
selic.show(5)

+-----+----------+
|SELIC|      Data|
+-----+----------+
|  9,9|2014-01-02|
|  9,9|2014-01-03|
|  9,9|2014-01-06|
|  9,9|2014-01-07|
|  9,9|2014-01-08|
+-----+----------+
only showing top 5 rows



In [0]:
# Trocando virgula por ponto
selic = selic.withColumn("SELIC", F.regexp_replace(F.col("SELIC"), "[,]", ".")) # trocando ',' por '.'
selic.show(5)

+-----+----------+
|SELIC|      Data|
+-----+----------+
|  9.9|2014-01-02|
|  9.9|2014-01-03|
|  9.9|2014-01-06|
|  9.9|2014-01-07|
|  9.9|2014-01-08|
+-----+----------+
only showing top 5 rows



In [0]:
# Adicione uma nova coluna contendo o último dia de cada mês
selicData = selic.withColumn("ultimo_dia", last_day("Data"))
# Filtrando pelo último dia de cada mês
selicData = selicData.filter(dayofmonth("Data") == dayofmonth("ultimo_dia"))
selicData.show(5)

+-----+----------+----------+
|SELIC|      Data|ultimo_dia|
+-----+----------+----------+
| 10.4|2014-01-31|2014-01-31|
|10.65|2014-02-28|2014-02-28|
|10.65|2014-03-31|2014-03-31|
| 10.9|2014-04-30|2014-04-30|
| 10.9|2014-06-30|2014-06-30|
+-----+----------+----------+
only showing top 5 rows



In [0]:
# Selecionando as colunas neessárias
selicFinal = selicData.select('Data', 'SELIC')
selicFinal.printSchema()

root
 |-- Data: date (nullable = true)
 |-- SELIC: string (nullable = true)



In [0]:
# trocando valor string para double
selicConcat = selicFinal.withColumn("SELIC", col("SELIC").cast("double")) 
display(selicConcat)

Data,SELIC
2014-01-31,10.4
2014-02-28,10.65
2014-03-31,10.65
2014-04-30,10.9
2014-06-30,10.9
2014-07-31,10.9
2014-09-30,10.9
2014-10-31,11.15
2014-12-31,11.65
2015-03-31,12.65


Databricks visualization. Run in Databricks to view.

In [0]:
selicConcat.printSchema()

root
 |-- Data: date (nullable = true)
 |-- SELIC: double (nullable = true)



# Concatenando Datasets

#### Ajustando os datasets

In [0]:
# Extraindo mês e ano da coluna de data
ifixReg = ifixConcat.withColumn("Mes_Ano", concat(month("Data"), lit("-"), year("Data")))
ifixReg = ifixReg.select('IFIX', 'Mes_Ano')
ifixReg.show(5)

+-------+-------+
|   IFIX|Mes_Ano|
+-------+-------+
|1613.76| 1-2013|
|1597.88| 2-2013|
|1579.32| 3-2013|
| 1549.0| 4-2013|
|1549.36| 5-2013|
+-------+-------+
only showing top 5 rows



In [0]:
# Extraindo mês e ano da coluna de data
cdiReg = cdiConcat.withColumn("Mes_Ano", concat(month("Data"), lit("-"), year("Data")))
cdiReg = cdiReg.select('CDI', 'Mes_Ano')
cdiReg.show(5)

+----+-------+
| CDI|Mes_Ano|
+----+-------+
|6.95| 1-2013|
|6.98| 2-2013|
|7.23| 4-2013|
|7.72| 5-2013|
|8.22| 7-2013|
+----+-------+
only showing top 5 rows



In [0]:
# Extraindo mês e ano da coluna de data
selicReg = selicConcat.withColumn("Mes_Ano", concat(month("Data"), lit("-"), year("Data")))
selicReg = selicReg.select('SELIC', 'Mes_Ano')
selicReg.show(5)

+-----+-------+
|SELIC|Mes_Ano|
+-----+-------+
| 10.4| 1-2014|
|10.65| 2-2014|
|10.65| 3-2014|
| 10.9| 4-2014|
| 10.9| 6-2014|
+-----+-------+
only showing top 5 rows



In [0]:
selicReg.printSchema()

root
 |-- SELIC: double (nullable = true)
 |-- Mes_Ano: date (nullable = true)



#### Dataset dos Indices

In [0]:
indices = ifixReg.join(cdiReg, "Mes_Ano", "inner").join(selicReg, "Mes_Ano", "inner")
indices.show(5)

+-------+-------+-----+-----+
|Mes_Ano|   IFIX|  CDI|SELIC|
+-------+-------+-----+-----+
| 1-2014|1282.04|10.27| 10.4|
| 2-2014|1330.42|10.59|10.65|
| 3-2014|1336.07|10.55|10.65|
| 4-2014|1350.76| 10.8| 10.9|
| 6-2014|1387.63| 10.8| 10.9|
+-------+-------+-----+-----+
only showing top 5 rows



In [0]:
indices.printSchema()

root
 |-- Mes_Ano: string (nullable = true)
 |-- IFIX: double (nullable = true)
 |-- CDI: double (nullable = true)
 |-- SELIC: double (nullable = true)



In [0]:
indices_date = indices.withColumn("Data", to_date("Mes_Ano", "M-yyyy"))
indices_date.printSchema()


root
 |-- Mes_Ano: string (nullable = true)
 |-- IFIX: double (nullable = true)
 |-- CDI: double (nullable = true)
 |-- SELIC: double (nullable = true)
 |-- Data: date (nullable = true)



In [0]:
indicesFinal = indices_date.select('Data', 'IFIX', 'CDI', 'SELIC')
indicesFinal.show(5)

+----------+-------+-----+-----+
|      Data|   IFIX|  CDI|SELIC|
+----------+-------+-----+-----+
|2014-01-01|1282.04|10.27| 10.4|
|2014-02-01|1330.42|10.59|10.65|
|2014-03-01|1336.07|10.55|10.65|
|2014-04-01|1350.76| 10.8| 10.9|
|2014-06-01|1387.63| 10.8| 10.9|
+----------+-------+-----+-----+
only showing top 5 rows



In [0]:
indicesFinal.printSchema()

root
 |-- Data: date (nullable = true)
 |-- IFIX: double (nullable = true)
 |-- CDI: double (nullable = true)
 |-- SELIC: double (nullable = true)



In [0]:
indicesFinal.count()

Out[276]: 85

In [0]:
display(indicesFinal)

Data,IFIX,CDI,SELIC
2014-01-01,1282.04,10.27,10.4
2014-02-01,1330.42,10.59,10.65
2014-03-01,1336.07,10.55,10.65
2014-04-01,1350.76,10.8,10.9
2014-06-01,1387.63,10.8,10.9
2014-07-01,1409.42,10.8,10.9
2014-09-01,1421.44,10.81,10.9
2014-10-01,1402.46,11.06,11.15
2014-12-01,1336.72,11.57,11.65
2015-03-01,1347.63,12.6,12.65


In [0]:
indicesFinal.printSchema()

root
 |-- Data: date (nullable = true)
 |-- IFIX: float (nullable = true)
 |-- CDI: float (nullable = true)
 |-- SELIC: float (nullable = true)



# Iniciando o Processo de Regressão

In [0]:
indicesFinal.describe().show()

+-------+------------------+-----------------+-----------------+
|summary|              IFIX|              CDI|            SELIC|
+-------+------------------+-----------------+-----------------+
|  count|                85|               85|               85|
|   mean|2319.9676470588242|9.484117647058808|9.499999999999984|
| stddev| 630.6641043992679| 4.09802893797379|4.105418892843248|
|    min|           1282.04|              1.9|              1.9|
|    max|            3360.0|            14.14|            14.15|
+-------+------------------+-----------------+-----------------+



In [0]:
#transformando os dados (linhas) em vetores
from pyspark.ml.feature import VectorAssembler  #importando a biblioteca responsável por criar o vetor a partir da coluna

assembler = VectorAssembler(inputCols=['CDI', 'SELIC'], outputCol='features')  #define o objeto para transformação
indicesRegression = assembler.transform(indicesFinal) #aplica a transformação
indicesRegression.printSchema()

root
 |-- Data: date (nullable = true)
 |-- IFIX: double (nullable = true)
 |-- CDI: double (nullable = true)
 |-- SELIC: double (nullable = true)
 |-- features: vector (nullable = true)



In [0]:
indicesRegression = indicesRegression.select(['features','IFIX'])
display(indicesRegression)

features,IFIX
"Map(vectorType -> dense, length -> 2, values -> List(10.27, 10.4))",1282.04
"Map(vectorType -> dense, length -> 2, values -> List(10.59, 10.65))",1330.42
"Map(vectorType -> dense, length -> 2, values -> List(10.55, 10.65))",1336.07
"Map(vectorType -> dense, length -> 2, values -> List(10.8, 10.9))",1350.76
"Map(vectorType -> dense, length -> 2, values -> List(10.8, 10.9))",1387.63
"Map(vectorType -> dense, length -> 2, values -> List(10.8, 10.9))",1409.42
"Map(vectorType -> dense, length -> 2, values -> List(10.81, 10.9))",1421.44
"Map(vectorType -> dense, length -> 2, values -> List(11.06, 11.15))",1402.46
"Map(vectorType -> dense, length -> 2, values -> List(11.57, 11.65))",1336.72
"Map(vectorType -> dense, length -> 2, values -> List(12.6, 12.65))",1347.63


Criando o Modelo de Regressão

In [0]:
train, test = indicesRegression.randomSplit([0.7, 0.3], seed=123)

In [0]:
train.show(5)

+-----------+-------+
|   features|   IFIX|
+-----------+-------+
|  [1.9,1.9]| 2782.1|
|  [1.9,1.9]|2794.88|
|  [1.9,1.9]|2870.15|
|[2.15,2.15]|2733.12|
|[2.15,2.15]|2806.28|
+-----------+-------+
only showing top 5 rows



In [0]:
from pyspark.ml.regression import LinearRegression  #biblioteca que contém o modelo de regressão

lr = LinearRegression(maxIter=10, labelCol='IFIX') # define o objeto a ser utilizado para regressão
lrModel = lr.fit(train)

In [0]:
#Coeficientes angulares e lineares (a e b) da reta de regressão
print(f'Intercepto: {lrModel.intercept}\nCoeficiente Angular: {lrModel.coefficients.values}')

Intercepto: 2812.8810682940625
Coeficiente Angular: [ 14296.34076984 -14320.67830101]


In [0]:
#print das estatísticas do modelo
modelsummary = lrModel.summary

print("Variância Explicada:", modelsummary.explainedVariance)
print('R_2: ', modelsummary.r2)
print('Erro médio quadrático: ',modelsummary.meanSquaredError)

modelsummary.residuals.show(5)


Variância Explicada: 227253.48827756057
R_2:  0.5567481728946275
Erro médio quadrático:  180926.54592828883
+------------------+
|         residuals|
+------------------+
|15.460240941359189|
| 28.24024094135939|
|103.51024094135937|
|-27.43537626450916|
| 45.72462373549115|
+------------------+
only showing top 5 rows



Realizando a Previsão Através do Modelo

In [0]:
modelsummary.predictions.show(5)

+-----------+-------+------------------+
|   features|   IFIX|        prediction|
+-----------+-------+------------------+
|  [1.9,1.9]| 2782.1|2766.6397590586407|
|  [1.9,1.9]|2794.88|2766.6397590586407|
|  [1.9,1.9]|2870.15|2766.6397590586407|
|[2.15,2.15]|2733.12| 2760.555376264509|
|[2.15,2.15]|2806.28| 2760.555376264509|
+-----------+-------+------------------+
only showing top 5 rows



In [0]:
display(modelsummary.predictions)

features,IFIX,prediction
"Map(vectorType -> dense, length -> 2, values -> List(1.9, 1.9))",2782.1,2766.6397590586407
"Map(vectorType -> dense, length -> 2, values -> List(1.9, 1.9))",2794.88,2766.6397590586407
"Map(vectorType -> dense, length -> 2, values -> List(1.9, 1.9))",2870.15,2766.6397590586407
"Map(vectorType -> dense, length -> 2, values -> List(2.15, 2.15))",2733.12,2760.555376264509
"Map(vectorType -> dense, length -> 2, values -> List(2.15, 2.15))",2806.28,2760.555376264509
"Map(vectorType -> dense, length -> 2, values -> List(2.65, 2.65))",2861.15,2748.3866106762384
"Map(vectorType -> dense, length -> 2, values -> List(3.4, 3.4))",2816.46,2730.1334622938325
"Map(vectorType -> dense, length -> 2, values -> List(3.65, 3.65))",2603.62,2724.0490794997045
"Map(vectorType -> dense, length -> 2, values -> List(4.15, 4.15))",2754.89,2711.880313911434
"Map(vectorType -> dense, length -> 2, values -> List(4.9, 4.9))",2791.98,2693.627165529028


In [0]:
# Grafico de linhas do IFIX + predição para comparação
predictionsDF = modelsummary.predictions
predictionsPlot = predictionsDF.join(ifixReg, "IFIX", "inner")
predictionsPlot = predictionsPlot.select('Mes_Ano', 'IFIX', 'prediction')
display(predictionsPlot)

Mes_Ano,IFIX,prediction
8-2020,2782.1,2766.6397590586407
9-2020,2794.88,2766.6397590586407
12-2020,2870.15,2766.6397590586407
7-2020,2733.12,2760.555376264509
6-2020,2806.28,2760.555376264509
4-2021,2861.15,2748.3866106762384
5-2021,2816.46,2730.1334622938325
4-2020,2603.62,2724.0490794997045
6-2021,2754.89,2711.880313911434
10-2019,2791.98,2693.627165529028


Databricks visualization. Run in Databricks to view.